In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input

from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

from glob import glob

import numpy as np

import matplotlib.pyplot as plt

In [2]:
# Load the pre-trained VGG16 model without the top (fully connected) layers
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [3]:
for layer in vgg.layers:
    layer.trainable=False
    
    

In [4]:
x=Flatten()(vgg.output)

In [5]:
#adding output layer
prediction=Dense(8,activation='softmax')(x)

In [6]:
#creating model
model = Model(inputs=vgg.input, outputs=prediction)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
#tell the model cost and optimization method to use

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], 
    run_eagerly = True
    )

In [9]:
#import image data generator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_datagen.flow_from_directory('C:\\Users\\Narayanan\\Tea Leaves\\tea sickness dataset',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=20)

Found 669 images belonging to 8 classes.


In [12]:
test_set = train_datagen.flow_from_directory('C:\\Users\\Narayanan\\Tea Leaves\\test',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=20)

Found 129 images belonging to 8 classes.


In [13]:
training_set.class_indices

{'Anthracnose': 0,
 'algal leaf': 1,
 'bird eye spot': 2,
 'brown blight': 3,
 'gray light': 4,
 'healthy': 5,
 'red leaf spot': 6,
 'white spot': 7}

In [14]:
#fit the model
r = model.fit_generator(training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
    )

D:\Program Files (x86)\ipykernel_6984\1968481864.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(training_set,


Epoch 1/30
34/34 [==============================] - 553s 16s/step - loss: 1.7082 - accuracy: 0.4155 - val_loss: 1.6316 - val_accuracy: 0.4031
Epoch 2/30
34/34 [==============================] - 552s 16s/step - loss: 0.9591 - accuracy: 0.6413 - val_loss: 1.4548 - val_accuracy: 0.5814
Epoch 3/30
34/34 [==============================] - 543s 16s/step - loss: 0.6742 - accuracy: 0.7444 - val_loss: 1.3995 - val_accuracy: 0.7442
Epoch 4/30
34/34 [==============================] - 553s 16s/step - loss: 0.4584 - accuracy: 0.8430 - val_loss: 1.3558 - val_accuracy: 0.7054
Epoch 5/30
34/34 [==============================] - 540s 16s/step - loss: 0.4768 - accuracy: 0.8221 - val_loss: 1.3836 - val_accuracy: 0.6977
Epoch 6/30
34/34 [==============================] - 540s 16s/step - loss: 0.3446 - accuracy: 0.8834 - val_loss: 1.4261 - val_accuracy: 0.7054
Epoch 7/30
34/34 [==============================] - 542s 16s/step - loss: 0.3135 - accuracy: 0.9013 - val_loss: 1.4296 - val_accuracy: 0.7364
Epoch 

In [16]:
model.save('vgg16final.h5')